In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Audio

In [ ]:
import numpy as np
from tqdm import tqdm_notebook
import librosa

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch import nn

In [ ]:
from zachary.datasets import AtemporalDataset, GANDataset, f0_from_stft_frame, pseudo_one_hot
from zachary.utils import get_torch_device, get_num_trainable_params
from zachary.weight_initializers import initialize_model

In [ ]:
BATCH_SIZE = 64
DEVICE = get_torch_device()

In [ ]:
dataset = AtemporalDataset()

In [ ]:
dataset[0][0].shape, dataset[0][1].shape

In [ ]:
dur = librosa.time_to_frames(10, sr=44100, hop_length=512, n_fft=1024)

In [ ]:
def get_data(size, i):
    specgram = np.empty((size, dataset[0][i].shape[0]), dtype='float32')
    for idx in range(size):
        specgram[idx] = dataset[idx][i]
    return specgram

In [ ]:
plt.rcParams['figure.figsize'] = (18, 4) 
specgram = get_data(dur, 0)

fig, (ax1) = plt.subplots(1, 1)
ax1.imshow(specgram.T, aspect='auto', origin='lower')
pass

In [ ]:
plt.rcParams['figure.figsize'] = (18, 4)
specgram = get_data(dur, 1)
    
fig, (ax1) = plt.subplots(1, 1)
ax1.imshow(specgram.T, aspect='auto', origin='lower')
pass

In [ ]:
def stft_to_signal(S, num_iters=15):
    S_T = S.T

    # Retrieve phase information
    phase = 2 * np.pi * np.random.random_sample(S_T.shape) - np.pi
    signal = None
    for idx in range(num_iters):
        D = S_T * np.exp(1j * phase)
        signal = librosa.istft(D, hop_length=512, win_length=1024)
        # don't calculate phase during the last iteration, because it will not be used.
        if idx < num_iters - 1:
            phase = np.angle(librosa.stft(signal, n_fft=1024, hop_length=512))

    return signal

In [ ]:
sig = stft_to_signal(get_data(dur, 0), num_iters=30)

In [ ]:
plt.rcParams['figure.figsize'] = (18, 4)
t = np.linspace(0, len(sig)/44100, len(sig))

fig, (ax1) = plt.subplots(1, 1)
ax1.plot(t, sig)
pass

In [ ]:
Audio(sig, rate=44100)

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.c1 = nn.Linear(513, 256)
        self.c2 = nn.Linear(256, 128)
        self.c3 = nn.Linear(128, 64)
        self.c4 = nn.Linear(64, 32)
        self.c5 = nn.Linear(32, 4)
        
    def forward(self, x):
        z = F.relu(self.c1(x))
        z = F.relu(self.c2(z))
        z = F.relu(self.c3(z))
        z = F.relu(self.c4(z))
        z = self.c5(z)
        
        return z

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.c1 = nn.Linear(4+128, 32)
        self.c2 = nn.Linear(32, 64)
        self.c3 = nn.Linear(64, 128)
        self.c4 = nn.Linear(128, 256)
        self.c5 = nn.Linear(256, 513)
        
    def forward(self, x):
        z = F.relu(self.c1(x))
        z = F.relu(self.c2(z))
        z = F.relu(self.c3(z))
        z = F.relu(self.c4(z))
        z = self.c5(z)
        
        return z

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = Encoder()
        self.decoder = Decoder()  

    def forward(self, x, f0):
        z = self.encoder(x)
        y = self.decoder(torch.cat([z, f0], 1))
        
        return y

In [ ]:
model = Autoencoder()
initialize_model(model)
print('\t', get_num_trainable_params(model))

In [ ]:
loss_fn = F.mse_loss

In [ ]:
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
data_loader = DataLoader(dataset, pin_memory=True, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)

### This is the training loop

In [ ]:
model.to(DEVICE)
model.train()
for i in range(2):
    batch = 1
    with tqdm_notebook(total=len(dataset)) as pbar:
        for absolute, f0 in data_loader:
            optimizer.zero_grad()
            absolute = absolute.to(DEVICE)
            f0 = f0.to(DEVICE)
            loss = loss_fn(model(absolute, f0), absolute)

            pbar.set_description(f'Epoch: {i + 1} - loss: {loss.data.cpu().numpy():.2E}')
            pbar.update(absolute.shape[0])

            batch += 1

            loss.backward()
            optimizer.step()

## Test performance

In [ ]:
from zachary.datasets import load_audio_file

In [ ]:
sample = torch.from_numpy(get_data(dur, 0)).to(DEVICE)
pitches = torch.from_numpy(get_data(dur, 1)).to(DEVICE)

In [ ]:
sample.shape, pitches.shape

In [ ]:
model.eval()
with torch.no_grad():
    sample_hat = model(sample, pitches)

sample_hat = sample_hat.cpu()
sample_hat_np = sample_hat.numpy()

In [ ]:
signal_hat = stft_to_signal(sample_hat_np, num_iters=30)

In [ ]:
plt.rcParams['figure.figsize'] = (18, 4)
t = np.linspace(0, len(sig)/44100, len(sig))

fig, (ax1) = plt.subplots(1, 1)
ax1.plot(t, signal_hat)
pass

In [ ]:
Audio(signal_hat, rate=44100)

In [ ]:
import scipy.interpolate as si

In [ ]:
def bspline(cv, n=100, degree=3, periodic=False):
    # If periodic, extend the point array by count+degree+1
    if degree < 1:
        raise ValueError('degree cannot be less then 1!')
    count = len(cv)

    if periodic:
        factor, fraction = divmod(count + degree + 1, count)
        cv = np.concatenate((cv,) * factor + (cv[:fraction],))
        count = len(cv)

    # If opened, prevent degree from exceeding count-1
    else:
        if count < degree + 1:
            raise ValueError('number of cvs must be higher than degree + 1')

    # Calculate knot vector
    if periodic:
        kv = np.arange(0 - degree, count + degree + degree - 1, dtype='int')
    else:
        kv = np.array([0] * degree + list(range(count - degree + 1)) + [count - degree] * degree, dtype='int')

    # Calculate query range
    u = np.linspace(periodic, (count - degree), n)

    # Calculate result
    arange = np.arange(len(u))
    points = np.zeros((len(u), cv.shape[1]))
    for i in range(cv.shape[1]):
        points[arange, i] = si.splev(u, (kv, cv[:, i], degree))

    return points


def sample_z(z_dims, mean, std, num_cv, resolution, degree, is_periodic):
    # Generates splines of random lengths in z_dims dimensions
    # num_cv = np.random.randint(64, 128)
    cv = np.random.normal(mean, std, (num_cv, z_dims))
    num_points = num_cv * resolution
    spline = bspline(cv, num_points, degree, is_periodic)
    return spline

In [ ]:
resolution = 25
num_cv = dur // resolution

In [ ]:
zs = sample_z(model.encoder.c5.out_features, 0., 2., num_cv, resolution, 2, True)

In [ ]:
zs_t = torch.from_numpy(zs.astype('float32')).to(DEVICE)
pitches = torch.from_numpy(get_data(zs_t.shape[0], 1)).to(DEVICE)
#pitches = torch.zeros_like(pitches)
#pitches[:, 60] = 1.

In [ ]:
model.eval()
with torch.no_grad():
    y = model.decoder(torch.cat([zs_t, pitches], 1))

y_hat = y.cpu()
y_hat_np = y_hat.numpy()

In [ ]:
s_hat = stft_to_signal(y_hat_np, num_iters=100)

In [ ]:
plt.rcParams['figure.figsize'] = (18, 4)
t = np.linspace(0, len(s_hat)/44100, len(s_hat))

fig, (ax1) = plt.subplots(1, 1)
ax1.plot(t, s_hat)
pass

In [ ]:
Audio(s_hat, rate=44100)

In [ ]:
gan_dataset = GANDataset(dataset, model.encoder, example_length=64, stft_hop_length=32)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm1d(128),

            nn.ConvTranspose1d(128, 64, 4, 4),
            nn.BatchNorm1d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.ConvTranspose1d(64, 32, 4, 4),
            nn.BatchNorm1d(32, 0.8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.ConvTranspose1d(32, 16, 4, 4),
        )

    def forward(self, z):
        img = self.conv_blocks(z)
        return img

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.conv_blocks = nn.Sequential(
            nn.Conv1d(16, 32, 4, 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),
            nn.BatchNorm1d(32, 0.8),
            
            nn.Conv1d(32, 64, 4, 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),
            nn.BatchNorm1d(64, 0.8),
            
            nn.Conv1d(64, 128, 4, 4),
        )

    def forward(self, img):
        validities = self.conv_blocks(img)
        return validities

In [ ]:
gen = Generator()
disc = Discriminator()
gen.to(DEVICE)
disc.to(DEVICE)
pass

In [ ]:
asd = torch.ones(1, 128, 4)
asd.to(DEVICE)
pass

In [ ]:
gen(asd).shape

In [ ]:
disc(gen(asd)).shape

In [ ]:
qwe = gan_dataset[0].transpose(0, 1).to(DEVICE)

model.eval()
with torch.no_grad():
    qwe = model.encoder(qwe).transpose(0, 1)

disc(qwe.unsqueeze(0)).shape